In [1]:
import pyspark

try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [2]:
import re

stop_words = ["a", "about", "above", "after", "all", "am", "an", "and", "any",\
                "are", "as", "at", "be", "because", "been", "before", "being",\
              "below", "tween", "bot", "by", "could", "did", "do", "does", "doing",\
              "down", "during", "each", "few", "for", "from", "further",\
              "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her",\
              "here", "here's", "hers", "herself", "him", "himself", "his", "how",\
              "how's", "i", "i'd", "i'll", "i'm", "i've", "u'if", "in", "into", "is",\
              "it", "it's", "its", "itself", "let's", "me", "more", "most",\
              "my", "myself", "of", "off", "on", "once",\
              "only", "or", "other", "ought", "our", "ours", "ourselves", "out",\
              "over", "own", "same", "she", "she'd", "she'll", "she's",\
              "should", "so", "some", "such", "than", "that","that's",\
              "the", "their", "theirs", "them", "themselves", "then", "there", "there's",\
              "these", "they", "they'd", "they'll", "they're", "they've", "this",\
              "those", "through", "to", "too", "under", "until", "up", "very", "was",\
             "we", "we'd", "we'll", "we're", "we've", "were",\
              "what", "what's", "when", "when's", "where", "where's", "which", "while",\
              "who", "who's", "whom", "why", "why's", "with", "would",\
              "you", "you'd", "you'll", "you're", "you've", "your", "yours",\
              "yourself", "yourselves"]

stop_words = set(stop_words)


def quitar_reviews_repetidos(reviewsRDD):
    reviews_sin_rep = reviewsRDD.map( lambda review: (review.Text, review) )\
                    .reduceByKey( lambda review1, review2: review1 )

    reviews_sin_rep = reviews_sin_rep.map(lambda x: x[1])

    return reviews_sin_rep

def preprocesar(review):
    texto = quitar_signos(review.Text)
    resumen = quitar_signos(review.Summary)

    #texto = texto.lower()
    #resumen = resumen.lower()

    texto_palabras = texto.split(" ")
    resumen_palabras = resumen.split(" ")
    
    texto = quitar_stopWords(texto_palabras)
    resumen = quitar_stopWords(resumen_palabras)

    estructura = [review.Id, review.Prediction, resumen+texto]

    return estructura


def quitar_signos(texto):
    text = ""
    if texto is not None:
        text = texto.replace("<br />", " ")
        text = re.sub("[^a-zA-Z' ]", ' ', text)

    return text


def quitar_stopWords(texto):
    text = []
    for palabra in texto:
        palabra = palabra.lower()
        if palabra not in stop_words and palabra != '':
            text.append(palabra)

    return text


In [3]:
import pyspark_csv as pycsv

trainRDD = sc.textFile('data_train.csv')
train_dataframe = pycsv.csvToDataFrame(sqlCtx, trainRDD, parseDate=False)
train = train_dataframe.rdd


train_sin_rep = quitar_reviews_repetidos(train)

train_preprocesado = train_sin_rep.map(preprocesar)

for x in train_preprocesado.take(1):
    for y in x:
        print y


279582
5
[u'love', u'healthy', u'delicious', u'tastes', u'great', u'spread', u'apple', u'banana', u'honey', u'adds', u'just', u'enough', u'sweetness', u'make', u'taste', u'like', u'guilty', u'pleasure', u'without', u'making', u'overly', u'sugary']


In [4]:
testRDD = sc.textFile('data_test.csv')
test_dataframe = pycsv.csvToDataFrame(sqlCtx, testRDD, parseDate=False)
test = test_dataframe.rdd


test_sin_rep = quitar_reviews_repetidos(test)

test_preprocesado = test_sin_rep.map(preprocesar)

for x in test_preprocesado.take(1):
#    print "x: ", x
    for y in x:
        print y


201541
3
[u'not', u'entirely', u'satisfied', u'even', u'though', u'shipping', u'super', u'fast', u'not', u'good', u'hoped', u'pretty', u'sure', u'received', u'pound', u'half', u'definitely', u'not', u'full', u'two', u'pounds', u'guess', u'fine', u"don't", u'see', u'finishing', u'given', u'flavor', u'decent', u'but', u'no', u'near', u'good', u'dried', u'papaya', u'can', u"didn't", u'seem', u'fresh', u'either', u'probably', u'wont', u'reordering', u'item', u'again']


In [16]:

def armar_palabras_con_sentimiento(nombre_archivo):
    palabras_con_sentimiento = []
    with open(nombre_archivo, 'r') as arch:
        for word in arch:
            word = word.rstrip("\n")
            palabras_con_sentimiento.append(word)

    return set(palabras_con_sentimiento)


def sentimiento(review):

    sentimiento = 0
    pos = []
    neg = []


    for i in range( len(review) ):
        word_ant = ""
        if i>0:
            word_ant = review[i-1]
        
        word = review[i]
        
        if word in palabras_positivas:
            if word_ant not in palabras_sentimiento_inverso:
                sentimiento += 1
                pos.append(word)
            else:
                sentimiento -= 1
                neg.append(word)

        if word in palabras_negativas:
            if word_ant not in palabras_sentimiento_inverso:
                sentimiento -= 1
                neg.append(word)
            else:
                sentimiento += 1
                pos.append(word)

    return [float(sentimiento), pos, neg]
#    return float(sentimiento)



palabras_positivas = armar_palabras_con_sentimiento("positive-words")
palabras_negativas = armar_palabras_con_sentimiento("negative-words")

palabras_sentimiento_inverso = ["not", "ain't", "bareley", "aren't", "can't", "couldn't", "didn't", "doesn't",\
                                "don't", "hadn't", "few", "hardly", "low", "merely", "neither", "never",\
                                "no", "nor", "nobody", "none", "nope", "nothing", "rarely", "seldom",\
                                "hasn't", "haven't", "isn't", "mustn't", "shan't", "shouldn't", "used",\
                                "wasn't", "weren't", "won't", "wouldn't", "zero"]


palabras_sentimiento_inverso = set(palabras_sentimiento_inverso)




In [6]:

def armar_palabras_con_sentimiento2(nombre_archivo):
    palabras_con_sentimiento = {}
    with open(nombre_archivo, 'r') as arch:
        for word in arch:
#            print "word1: ", word
            word = word.rstrip("\n")
#            word = word.rstrip("\t")
#            print "word2: ", word
            word = word.split("\t")
#            print "word: ", word
            palabras_con_sentimiento[word[0]] = int(word[1])

    return palabras_con_sentimiento

def sentimiento2(review):
    sentimiento = 0
    words = []
    
    for word in review:
        if word in palabras_sentimiento:
            sentimiento += palabras_sentimiento[word]
            words.append(word)
    
    return [sentimiento, words]


palabras_sentimiento = armar_palabras_con_sentimiento2("AFINN-96.txt")

#print "dict: ", palabras_sentimiento["aboard"]



In [14]:
## ARMAR PALABRAS CON SENTIMIENTO CON PESOS

def armar_palabras_con_sentimiento(nombre_archivo_entrada, nombre_archivo_salida, valor_default):
#    palabras_con_sentimiento = []
    with open(nombre_archivo_salida, 'w') as arch_salida:
        with open(nombre_archivo_entrada, 'r') as arch_entrada:
            for word in arch_entrada:
                s = ""
                word = word.rstrip("\n")
                
                if word in palabras_sentimiento:
                    s = word + " " + str(palabras_sentimiento[word]) + "\n"
                else:
                    s = word + " " + valor_default + "\n"

                arch_salida.write(s)
#                palabras_con_sentimiento.append(word)

    return

palabras_positivas = armar_palabras_con_sentimiento("positive-words", "positive-words-pesos", "1")
palabras_negativas = armar_palabras_con_sentimiento("negative-words", "negative-words-pesos", "-1")



In [9]:
def armar_palabras_con_sentimiento3(nombre_archivo):
    palabras_con_sentimiento = {}
    with open(nombre_archivo, 'r') as arch:
        for word in arch:
#            print "word1: ", word
            word = word.rstrip("\n")
#            word = word.rstrip("\t")
#            print "word2: ", word
            word = word.split(" ")
#            print "word: ", word
            palabras_con_sentimiento[word[0]] = int(word[1])

    return palabras_con_sentimiento

def sentimiento3(review):

    sentimiento = 0
    pos = []
    neg = []


    for i in range( len(review) ):
        word_ant = ""
        if i>0:
            word_ant = review[i-1]
        
        word = review[i]
        
        if word in palabras_pos:
            if word_ant not in palabras_sentimiento_inverso:
                sentimiento += palabras_pos[word]
                pos.append(word)
            else:
                sentimiento -= palabras_pos[word]
                neg.append(word)

        if word in palabras_neg:
            if word_ant not in palabras_sentimiento_inverso:
                sentimiento += palabras_neg[word]
                neg.append(word)
            else:
                sentimiento -= palabras_neg[word]
                pos.append(word)

    return [float(sentimiento), pos, neg]
#    return float(sentimiento)


palabras_pos = armar_palabras_con_sentimiento3("positive-words-pesos")
palabras_neg = armar_palabras_con_sentimiento3("negative-words-pesos")




In [10]:
## PARA NORMALIZAR DATOS
from math import sqrt


def obtener_sent(review, sentimiento):
    texto = review[2]
    sent = sentimiento(texto)
#    print "sent: ", sent[0]
    return (1, sent[0])

sentimientos = train_preprocesado.map(lambda review: obtener_sent(review, sentimiento3))

suma_sent = sentimientos.reduceByKey(lambda x, y: x+y)

suma_sent = suma_sent.collect()
suma_sent = suma_sent[0][1]

cant_sent = train_preprocesado.count()

promedio_sent = float(suma_sent)/cant_sent

print "suma_sent: ", suma_sent
print "cant_sent: ", cant_sent
print "promedio_sent: ", promedio_sent


ssd = sentimientos.map(lambda x: (1, ( x[1] - promedio_sent )**2 ) ).reduceByKey(lambda x, y: x+y)
ssd = ssd.collect()
ssd = ssd[0][1]

varianza = ssd / (cant_sent-1)
desv_std = sqrt(varianza)

print "desv_std: ", desv_std


suma_sent:  1860030.0
cant_sent:  229077
promedio_sent:  8.11967155149
desv_std:  7.7588113562


In [12]:
from math import sqrt

def distancia(texto1_estruct, texto2_estruct, sentimiento):
    texto1 = texto1_estruct[2]
    texto2 = texto2_estruct[2]

    sent1 = sentimiento(texto1)
    sent1 = sent1[0]

    sent2 = sentimiento(texto2)
    sent2 = sent2[0]

    ## NORMALIZO
#    sent1 = (sent1 - promedio_sent) / desv_std
#    sent2 = (sent2 - promedio_sent) / desv_std

    dist = sqrt( (sent1 - sent2)**2 )

    beta = 2
    if dist == 0:
        peso = float("inf")
    else:
        peso = float(1) / dist**beta

#    return [texto2_estruct[0], texto2_estruct[1], dist]
    return [texto2_estruct[0], texto2_estruct[1], peso]


def knn(texto_estruct, k, sentimiento):

    dist = train_preprocesado.map(lambda vecino: distancia(texto_estruct, vecino, sentimiento))
#    k_mas_cercanos = dist.takeOrdered(k, key=lambda x: x[2])

    ## PARA DIST2
    k_mas_cercanos = dist.takeOrdered(k, key=lambda x: -x[2])

    return k_mas_cercanos

def predecir(k_mas_cercanos):
    prediccion_total = 0
    for cercano in k_mas_cercanos:
        prediccion_total += cercano[1]

    return float(prediccion_total) / len(k_mas_cercanos)

cant_ej = 10
test = test_preprocesado.take(cant_ej)
#for t in test:
#    print "t: ", t

## PREDECIR VARIOS
error = 0
correct = 0
for test_pred in test:
    print "TEST_PRED: ", test_pred[0:2]
    k_mas_cercanos = knn(test_pred, 3, sentimiento3)
    prediccion = predecir(k_mas_cercanos)

    print "PRED: ", prediccion

    if prediccion == test_pred[1]:
        correct += 1
    error += (prediccion - test_pred[1])**2

error = sqrt(error)
print "ERROR: ", error

print "accuracy: ", ( float(correct)/cant_ej ) * 100

## PREDECIR UNO
"""
test_pred = test[5]

print "test: ", test_pred
print " "

print "test-sent: ", sentimiento(test_pred[2])
print " "


k_mas_cercanos = knn(test_pred, 10, sentimiento)
print "k_mas_cercanos: ", k_mas_cercanos

prediccion = predecir(k_mas_cercanos)

print " "
print "PRED: ", prediccion


k_mas_cercanos = knn(test_pred, 10, sentimiento2)
print "k_mas_cercanos2: ", k_mas_cercanos

prediccion = predecir(k_mas_cercanos)

print " "
print "PRED2: ", prediccion
"""



TEST_PRED:  [201541, 3]
PRED:  5.0
TEST_PRED:  [6911, 2]
PRED:  5.0
TEST_PRED:  [461853, 5]
PRED:  4.66666666667
TEST_PRED:  [563294, 5]
PRED:  4.66666666667
TEST_PRED:  [471402, 5]
PRED:  5.0
TEST_PRED:  [139618, 5]
PRED:  5.0
TEST_PRED:  [248946, 5]
PRED:  3.66666666667
TEST_PRED:  [163663, 1]
PRED:  3.33333333333
TEST_PRED:  [140792, 5]
PRED:  5.0
TEST_PRED:  [340744, 1]
PRED:  1.66666666667
TEST_PRED:  [278204, 4]
PRED:  5.0
TEST_PRED:  [165496, 1]
PRED:  4.33333333333
TEST_PRED:  [415793, 5]
PRED:  4.33333333333
TEST_PRED:  [268341, 4]
PRED:  4.66666666667
TEST_PRED:  [371580, 5]
PRED:  3.33333333333
TEST_PRED:  [538393, 4]
PRED:  3.66666666667
TEST_PRED:  [415317, 5]
PRED:  5.0
TEST_PRED:  [524561, 1]
PRED:  1.0
TEST_PRED:  [55501, 3]
PRED:  4.66666666667
TEST_PRED:  [545857, 5]
PRED:  4.0
TEST_PRED:  [64276, 5]
PRED:  4.33333333333
TEST_PRED:  [549022, 5]
PRED:  5.0
TEST_PRED:  [402962, 4]
PRED:  3.66666666667
TEST_PRED:  [514329, 5]
PRED:  2.66666666667
TEST_PRED:  [549413, 4]


'\ntest_pred = test[5]\n\nprint "test: ", test_pred\nprint " "\n\nprint "test-sent: ", sentimiento(test_pred[2])\nprint " "\n\n\nk_mas_cercanos = knn(test_pred, 10, sentimiento)\nprint "k_mas_cercanos: ", k_mas_cercanos\n\nprediccion = predecir(k_mas_cercanos)\n\nprint " "\nprint "PRED: ", prediccion\n\n\nk_mas_cercanos = knn(test_pred, 10, sentimiento2)\nprint "k_mas_cercanos2: ", k_mas_cercanos\n\nprediccion = predecir(k_mas_cercanos)\n\nprint " "\nprint "PRED2: ", prediccion\n'

In [103]:
## CALCULAR SENTIMIENTO PARA CADA PREDICCION

def foo(review):
    pred = review[1]
    texto = review[2]
    sent = sentimiento3(texto)
    
    if pred == 5 and sent[0] == 0:
        print "id: ", review[0]
        print "texto: ", texto
    
    return ( pred, [sent[0]] )

pred_sent = train_preprocesado.map(foo).reduceByKey( lambda a, b: a + b )

pred_sent = pred_sent.collect()

for x in pred_sent:
    print "PRED: ", x[0]
    print "SENT: ", set(x[1])



PRED:  5
SENT:  set([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 79.0, 81.0, 83.0, 84.0, 89.0, 90.0, 105.0, 106.0, 109.0, 119.0, 142.0, -2.0, -44.0, -34.0, -33.0, -30.0, -29.0, -23.0, -22.0, -21.0, -20.0, -19.0, -18.0, -17.0, -16.0, -15.0, -14.0, -13.0, -12.0, -11.0, -10.0, -9.0, -8.0, -7.0, -6.0, -5.0, -4.0, -3.0, -1.0])
PRED:  1
SENT:  set([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 30.0, 31.0, 32.0, 35.0, 36.0, 37.0, 40.0, 41.0, 43.0, 45.0, -80.0, -79.0, 50.0, 54.

In [27]:
## CALCULAR SENTIMIENTO PARA UN REVIEW ESPECIFICO

reviewRDD = sc.textFile("review")
review_dataframe = pycsv.csvToDataFrame(sqlCtx, reviewRDD, parseDate=False)
review = review_dataframe.rdd

#test_sin_rep = quitar_reviews_repetidos(test)

review_preprocesado = review.map(preprocesar)

review = review_preprocesado.collect()
print "review: ", review

print " "
print "sentimiento: ", sentimiento(review[0][2])

print " "
print "sentimiento2: ", sentimiento2(review[0][2])

print " "
print "sentimiento3: ", sentimiento3(review[0][2])



 review:  [[412489, 5, [u'excellent', u'quality', u'first', u'adopted', u'dog', u'vast', u'amount', u'research', u'find', u'truly', u'best', u'feed', u'chihuahua', u'mix', u'reading', u'mixed', u'reviews', u'whether', u'small', u'dogs', u'benefit', u'high', u'protein', u'diet', u'not', u'decided', u'just', u'try', u'little', u'trouble', u'digestion', u'transitioning', u'food', u'eating', u'royal', u'canin', u'prior', u'eating', u'seven', u'months', u'now', u'phenomenal', u'great', u'muscle', u'tone', u'energy', u'coat', u'silky', u'shinny', u'coarse', u'dull', u'yes', u'pricey', u'side', u'but', u'if', u'closely', u'watch', u'put', u'body', u'humans', u"wouldn't", u'want', u'best', u'friend', u'benefit', u'great', u'food']]]
 
sentimiento:  [4.0, [u'excellent', u'best', u'benefit', u'phenomenal', u'great', u'best', u'benefit', u'great'], [u'trouble', u'coarse', u'dull', u'pricey']]
 
sentimiento2:  [19, [u'excellent', u'best', u'benefit', u'trouble', u'great', u'yes', u'want', u'best',

In [40]:

def bow(review):
    texto = review[2]
    frec = {}
    for word in texto:
        if word in frec:
            frec[word] += 1
        else:
            frec[word] = 1

    review.append(frec)
    return review
#    return frec

def frec_por_doc(review):
    texto = set(review[2])
    words = []
    for word in texto:
        words.append( (word, 1) )

    return words


review_frec = review_preprocesado.map(bow)
#print "frec:", review_frec.collect()

frecs_por_docs = review_preprocesado.flatMap(frec_por_doc).reduceByKey( lambda x,y: x+y )
frecs_por_docs = frecs_por_docs.collect()

frec = {}

for word in frecs_por_docs:
    frec[word[0]] = word[1]

print " "
print "frecs_por_docs: ", frec


cant_reviews = review_preprocesado.count()








 
frecs_por_docs:  {u'neighbors': 1, u'just': 1, u"don't": 1, u'money': 1, u'go': 1, u'yes': 1, u'find': 1, u'chihuahua': 1, u'earned': 1, u'less': 1, u'royal': 1, u'wasting': 1, u'shinny': 1, u'holiday': 1, u'worth': 1, u'food': 1, u'watch': 1, u'truly': 1, u'amazon': 1, u'closely': 1, u'not': 2, u'trouble': 1, u'now': 1, u'vast': 1, u'like': 1, u'gift': 1, u'try': 1, u'benefit': 1, u'overflowing': 1, u'small': 1, u'tower': 1, u'side': 1, u'energy': 1, u'hard': 1, u'humans': 1, u'decided': 1, u'best': 1, u'silky': 1, u'phenomenal': 1, u'research': 1, u'reading': 1, u'body': 1, u'full': 1, u'eating': 1, u'put': 2, u'appears': 1, u'great': 1, u'goods': 1, u"wouldn't": 1, u'months': 1, u'times': 1, u'kosher': 1, u'prior': 1, u'amount': 1, u'mixed': 1, u'makes': 1, u'first': 2, u'feed': 1, u'fantastic': 1, u'digestion': 1, u'tone': 1, u'feel': 1, u'canin': 1, u'tiny': 1, u'array': 1, u'quality': 1, u'little': 1, u'negative': 1, u'long': 1, u'next': 1, u'wonderful': 1, u'coarse': 1, u'take